In [36]:
import pandas as pd
import sys
import requests
from xlsxwriter.utility import xl_rowcol_to_cell
sys.path.append(r'/home/jupyter/reusable_code')
import google_api_functions as gaf
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)


from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
import itertools
from sklearn.metrics import mean_squared_error
import numpy as np


from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Set up modules for Google functionality
from google.cloud import bigquery # To run BQ statements
from google_auth_oauthlib import flow # To authorise as user
from googleapiclient.discovery import build # To pull in from sheets, slides etc. API
from google.auth.transport.requests import Request
from google.cloud.bigquery import magics
import google

from datetime import datetime

In [37]:
def mean_absolute_percentage_error(actual, estimated): 
    actual, estimated = np.array(actual), np.array(estimated)
    return np.mean(np.abs((actual - estimated) / actual)) 

In [38]:
creds=gaf.Authenticate_Google(r"/home/jupyter/reusable_code/") #GAF is a package steve created with a list of useful functions
bq = bigquery.Client(project='itv-bde-analytics-dev',credentials=creds) #Apply credentials to BQ client "bq"

In [39]:
# Read in the FTS table created in BQ

query="""
select *
from `itv-bde-analytics-dev.britbox_sandbox.SB_4hourly_FTS`
;
"""
FTS_df = bq.query(query).to_dataframe()
FTS_df

In [40]:
# Change the column names (was relevant when using steves code)
FTS_df = FTS_df.rename(columns={ 'starting_hour':'reporting date','FTS':'N'})

# Indexes the date column 
FTS_df = FTS_df.set_index("reporting date")


In [41]:
# Order the data
FTS_df = FTS_df.sort_index()
FTS_df.tail()

In [45]:
# Alternative way of generating loop
import itertools
p_values = range(0,3)
d_values = range(0,2)
q_values = range(0,2)
s_values=[6,0] #Weekly, daily, none
pdq = list(itertools.product(p_values, d_values, q_values))
PDQ = list(itertools.product(p_values, d_values, q_values,s_values))
arima_combos=list(itertools.product(pdq,PDQ))
arima_combos=[i for i in arima_combos if i[0]!=(0,0,0)] # Remove all the 0,0,0 combos
print(len(pdq))
print(len(arima_combos))

In [76]:
mystr='Hello {0} and {0}!'.format('Sammy','Steve')
cmystr

In [79]:
mylist=['A','B','C']

for n,val in enumerate(mylist):
    print(n,val)

In [61]:
ARIMA_params={i:(['ar.L{}'.format(x+1) for x in range(0,i[0])],['ma.L{}'.format(x+1) for x in range(0,i[2])]) for i in pdq}
ARIMA_params

In [71]:
all_params=[]
for i in arima_combos:
    [all_params.append(item) for sublist in [x for x in ARIMA_params[i[0]]]+[y for y in SARIMA_params[i[1]]]+[['sigma2']] for item in sublist]
start_params=pd.Series({x:0.01 for x in set(all_params)})
start_params

In [75]:
keep_paramids = pd.Series([item for sublist in [x for x in ARIMA_params[(1,0,0)]]+[y for y in SARIMA_params[(1,0,0,6)]]+[['sigma2']] for item in sublist])
start_params.filter(items=keep_paramids)

In [8]:
train,test = FTS_df['2019-11-11 00:00:00':'2020-10-31 20:00:00'], FTS_df['2020-11-01 00:00:00':'2020-11-16 20:00:00']
errors=[]
errors2=[]
predictions = list()

# Initialise best model dictionary
best_model={'MAPE':999999999.0}

# Generate dictionary of the parameters that will be needed for each iteration of the model
ARIMA_params={i:(['ar.L{}'.format(x+1) for x in range(0,i[0])],['ma.L{}'.format(x+1) for x in range(0,i[2])]) for i in pdq}
SARIMA_params={i:(['ar.S.L{}'.format((x+1)*i[3]) for x in range(0,i[0])],['ma.S.L{}'.format((x+1)*i[3]) for x in range(0,i[2])]) for i in PDQ}

all_params=[]
for i in arima_combos:
    [all_params.append(item) for sublist in [x for x in ARIMA_params[i[0]]]+[y for y in SARIMA_params[i[1]]]+[['sigma2']] for item in sublist]
start_params=pd.Series({x:0.01 for x in set(all_params)})


for n,i in enumerate(reversed(arima_combos)):
    try:
        keep_paramids = pd.Series([item for sublist in [x for x in ARIMA_params[i[0]]]+[y for y in SARIMA_params[i[1]]]+[['sigma2']] for item in sublist])
        if i[0]!=(0,0,0):
            if i[1][3]==0: 
                model=ARIMA(train,order=i[0],freq='4H') # Initialise ARIMA class

                print('Starting ARIMA for {}  at {}'.format(i,datetime.now().strftime("%d/%m/%Y %H:%M:%S")))
                model_fit = model.fit(disp=0)  # Actually fit the ARIMA

                print('Finished ARIMA for {}  at {}'.format(i,datetime.now().strftime("%d/%m/%Y %H:%M:%S")))
                pred_y = model_fit.forecast(steps=len(test))[0]# create a forecast for the "expected" values on the Test Dataset

            else:
         #   try:

                model=SARIMAX(train, order=i[0], seasonal_order=i[1],freq='4h', simple_differencing=True) # Initialise SARIMAX class

                print('Starting SARIMAX for {} at {}'.format(i,datetime.now().strftime("%d/%m/%Y %H:%M:%S")))
                #print(start_params.filter(items=keep_paramids))
                # Actually fit the SARIMAX
                init_params=start_params.filter(items=keep_paramids)
                print(init_params)
                model_fit = model.fit(disp=0,low_memory=True,start_params=init_params) #start params are filtered and sorted according to keep_paramids
                #model_fit = model.fit(disp=0,low_memory=True) # Actually fit the SARIMAX
                print('Finished SARIMAX for {}  at {}'.format(i,datetime.now().strftime("%d/%m/%Y %H:%M:%S")))
                print(model_fit.params)

                pred_y = model_fit.forecast(steps=len(test)) # create a forecast for the "expected" values on the Test Dataset

            predictions.append(pred_y)

            error = mean_squared_error(test,pred_y)
            error2 = mean_absolute_percentage_error(test,pred_y)

            if error2*100<best_model['MAPE']:

                print('{} was this model''s error, and {} is the previous best so model will update'.format(error2*100,best_model['MAPE']))
                # Update view of what best model is
                best_model={'model':model_fit\
                           ,'modelParams':model_fit.params\
                           ,'modelParamsConfInt':model_fit.conf_int()\
                            ,'ARIMA':i[0]\
                            ,'SARIMA':i[1]\
                            ,'RMSE':np.sqrt(error)
                            ,'MAPE':error2*100
                           }
                # Update Start Params to hopefully optimise future loops
                start_params=model_fit.params.combine_first(start_params)
            errors.append(error)
            errors2.append(error2)
            print(i[0],i[1], np.sqrt(error), error2*100)
    except:
        print('Could not fit model {}'.format(i))

In [80]:
best_model


In [53]:
print(i[0],i[1])
model=SARIMAX(train, order=(0,1,1), seasonal_order=(1,1,1,42), simple_differencing=True,freq='4H')
            
timenow=datetime.now()
print('Starting SARIMAX for {} at {}'.format(i,timenow.strftime("%d/%m/%Y %H:%M:%S")))
model_fit = model.fit(disp=0,low_memory=True,freq='4H')
timenow=datetime.now()
print('Finished SARIMAX for {}  at {}'.format(i,timenow.strftime("%d/%m/%Y %H:%M:%S")))
            

#help(model_fit)
print(model_fit.model) # Model instance
print(model_fit.params)# Actual params
model_fit.conf_int() # Confidence intervals



In [49]:
type(model_fit.params)

In [54]:
last_params=model_fit.params

model=SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,42), simple_differencing=True,freq='4H')
            
timenow=datetime.now()
print('Starting SARIMAX for {} at {}'.format(i,timenow.strftime("%d/%m/%Y %H:%M:%S")))
model_fit = model.fit(disp=0,low_memory=True,freq='4H',start_params=last_params)
timenow=datetime.now()
print('Finished SARIMAX for {}  at {}'.format(i,timenow.strftime("%d/%m/%Y %H:%M:%S")))
            

#help(model_fit)
print(model_fit.model) # Model instance
print(model_fit.params)# Actual params
model_fit.conf_int() # Confidence intervals


In [55]:
last_params=model_fit.params
print(i[0],i[1])
model=SARIMAX(train, order=(0,1,1), seasonal_order=(1,1,1,42), simple_differencing=True,freq='4H')
            
timenow=datetime.now()
print('Starting SARIMAX for {} at {}'.format(i,timenow.strftime("%d/%m/%Y %H:%M:%S")))
model_fit = model.fit(disp=0,low_memory=True,freq='4H',start_params=last_params)
timenow=datetime.now()
print('Finished SARIMAX for {}  at {}'.format(i,timenow.strftime("%d/%m/%Y %H:%M:%S")))
            

#help(model_fit)
print(model_fit.model) # Model instance
print(model_fit.params)# Actual params
model_fit.conf_int() # Confidence intervals


In [60]:
start_params_dict={'ar.L1':0.00,\
                   'ma.L1':0.00,\
'ar.S.L42':0.00,\
'ma.S.L42':0.00,\
'sigma2':0.00}

In [62]:
start_params=pd.Series(start_params_dict)
start_params

In [65]:
new_params_dict={'ar.L1':1.20,\
                   'ma.L1':0.99}
new_params=pd.Series(new_params_dict)

In [67]:
start_params=new_params.combine_first(start_params)

In [71]:
start_params=pd.Series({'ar.L1':0.00,\
                   'ma.L1':0.00,\
'ar.S.L42':0.00,\
'ma.S.L42':0.00,\
'sigma2':0.00})

In [72]:
start_params

In [75]:
i=((0,1,1),(1,1,0,6))

In [84]:
[x[y for y in x] for x in i]]

In [93]:
paramids=pd.Series(['ar.L1','i1','ma.L1','ar.S.L42','i2','ma.S.L42','sigma2'])
hyperparameters=pd.Series([item for sublist in i for item in sublist])
keep_paramids=paramids[hyperparameters>0]
start_params.filter(items=keep_paramids)

In [95]:
keep_paramids.reindex()

In [143]:
pdq

In [145]:
p_values = range(0,3)
d_values = range(0,2)
q_values = range(0,3)
s_values=[42,6,0] #Weekly, daily, none
pdq = list(itertools.product(p_values, d_values, q_values))
PDQ = list(itertools.product(p_values, d_values, q_values,s_values))

In [182]:
ARIMA_params={i:(['ar.L{}'.format(x+1) for x in range(0,i[0])],['ma.L{}'.format(x+1) for x in range(0,i[2])]) for i in pdq}
SARIMA_params={i:(['ar.S.L{}'.format((x+1)*i[3]) for x in range(0,i[0])],['ma.S.L{}'.format((y+1)*i[3]) for y in range(0,i[2])]) for i in PDQ}

ARIMA_params

In [183]:
SARIMA_params={i:(['ar.S.L{}'.format((x+1)*i[3]) for x in range(0,i[0])],['ma.S.L{}'.format((y+1)*i[3]) for y in range(0,i[2])]) for i in PDQ}
SARIMA_params

In [181]:
train,test = FTS_df['2019-11-11 00:00:00':'2020-10-31 20:00:00'], FTS_df['2020-11-01 00:00:00':'2020-11-16 20:00:00']           
errors=[]
errors2=[]
predictions = list()

# Initialise best model dictionary
best_model={'MAPE':999999999.0}

# Generate dictionary of the parameters that will be needed for each iteration of the model
ARIMA_params={i:(['ar.L{}'.format(x+1) for x in range(0,i[0])],['ma.L{}'.format(x+1) for x in range(0,i[2])]) for i in pdq}
SARIMA_params={i:(['ar.S.L{}'.format((x+1)*i[3]) for x in range(0,i[0])],['ma.S.L{}'.format((x+1)*i[3]) for x in range(0,i[2])]) for i in PDQ}

all_params=[]
for i in arima_combos:
    [all_params.append(item) for sublist in [x for x in ARIMA_params[i[0]]]+[y for y in SARIMA_params[i[1]]]+[['sigma2']] for item in sublist]
start_params=pd.Series({x:0.00 for x in set(all_params)})

for i in reversed(arima_combos):
    keep_paramids = pd.Series([item for sublist in [x for x in ARIMA_params[i[0]]]+[y for y in SARIMA_params[i[1]]]+[['sigma2']] for item in sublist])
    print(keep_paramids)
    print(start_params.filter(items=keep_paramids))

In [178]:
train,test = FTS_df['2019-11-11 00:00:00':'2020-10-31 20:00:00'], FTS_df['2020-11-01 00:00:00':'2020-11-16 20:00:00']           
errors=[]
errors2=[]
predictions = list()

# Initialise best model dictionary
best_model={'MAPE':999999999.0}

# Generate dictionary of the parameters that will be needed for each iteration of the model
ARIMA_params={i:(['ar.L{}'.format(x+1) for x in range(0,i[0])],['ma.L{}'.format(x+1) for x in range(0,i[2])]) for i in pdq}
SARIMA_params={i:(['ar.S.L{}'.format((x+1)*i[3]) for x in range(0,i[0])],['ma.S.L{}'.format((x+1)*i[3]) for x in range(0,i[2])]) for i in PDQ}

all_params=[]
for i in arima_combos:
    [all_params.append(item) for sublist in [x for x in ARIMA_params[i[0]]]+[y for y in SARIMA_params[i[1]]]+[['sigma2']] for item in sublist]
start_params=pd.Series({x:0.00 for x in set(all_params)})
start_params


In [199]:
ser1=pd.Series(['A','B','C'])
ser2=pd.Series([2,1])


In [200]:
ser1.filter(items=ser2)

In [193]:
ser1

In [194]:
ser2

In [12]:

           
           
model=SARIMAX(train, order=(0, 0, 1), seasonal_order=(1, 0, 1, 42),freq='4h', simple_differencing=True) # Initialise SARIMAX class
print('Starting SARIMAX for {} at {}'.format(i,datetime.now().strftime("%d/%m/%Y %H:%M:%S")))
model_fit = model.fit(disp=0,low_memory=True) #start params are filtered and sorted according to keep_paramids
print('Finished SARIMAX for {}  at {}'.format(i,datetime.now().strftime("%d/%m/%Y %H:%M:%S")))
print(model_fit.params)

pred_y = model_fit.forecast(steps=len(test)) # create a forecast for the "expected" values on the Test Dataset

predictions.append(pred_y)

error = mean_squared_error(test,pred_y)
error2 = mean_absolute_percentage_error(test,pred_y)

In [14]:
train.plot()

In [19]:
Upper_Bound=train.mean()+(3*train.std())
Upper_Bound

In [30]:
train[train['N']>Upper_Bound[0]].shape

In [27]:
type(Upper_Bound[0])

In [35]:
Upper_Bound=train.mean()+(3*train.std())
train['N2'] = np.where(train['N']>Upper_Bound[0], Upper_Bound[0],train['N'])

In [33]:
train.plot()